In [50]:
import pandas as pd
import os

In [51]:
rows = [
    [65.8, 112, 30, 30, "N"],
    [71.5, 136, 19, 31, "N"],
    [69.4, 153, 45, 29, "N"],
    [68.2, 142, 22, 28, "Y"],
    [67.8, 144, 29, 24, "Y"],
    [68.7, 123, 50, 26, "N"],
    [69.8, 141, 51, 22, "Y"],
    [70.1, 136, 23, 20, "Y"],
    [67.9, 112, 17, 19, "N"],
    [66.8, 120, 39, 31, "N"],
]
columns = ["Height_in", "Weight_lb", "Age_yr", "Grip_kg", "Frailty"]
df = pd.DataFrame(rows, columns=columns)

In [52]:
os.makedirs("data_raw", exist_ok=True)


In [53]:
df.to_csv('/content/data_raw/data.csv',index=False)

In [54]:
df["Height_m"] = df["Height_in"] * 0.0254
df["Weight_kg"] = df["Weight_lb"] * 0.45359237
df = df.drop(columns=["Height_in", "Weight_lb"])
df

,Age_yr,Grip_kg,Frailty,Height_m,Weight_kg
0,30,30,N,1.67132,50.802345
1,19,31,N,1.81610,61.688562
2,45,29,N,1.76276,69.399633
3,22,28,Y,1.73228,64.410117
4,29,24,Y,1.72212,65.317301
5,50,26,N,1.74498,55.791862
6,51,22,Y,1.77292,63.956524
7,23,20,Y,1.78054,61.688562
8,17,19,N,1.72466,50.802345
9,39,31,N,1.69672,54.431084


In [55]:
df["BMI"] = (df["Weight_kg"] / (df["Height_m"]**2)).round(2)

In [56]:
def age_group(age):
    if age < 30: return "<30"
    elif 30 <= age <= 45: return "30–45"
    elif 46 <= age <= 60: return "46–60"
    else: return ">60"

df["AgeGroup"] = df["Age_yr"].apply(age_group)

In [57]:
df["Frailty_binary"] = df["Frailty"].map({"Y":1, "N":0}).astype("int8")
df = pd.get_dummies(df, columns=["AgeGroup"])

In [58]:
summary = df.describe().T[["mean","50%","std"]].rename(columns={"50%":"median"})


In [66]:
os.makedirs("Results", exist_ok=True)
summary.to_markdown('/content/Results/findings.md')

In [59]:
df=df.drop(columns=['Age_yr','Frailty'])

In [60]:
df

,Grip_kg,Height_m,Weight_kg,BMI,Frailty_binary,AgeGroup_30–45,AgeGroup_46–60,AgeGroup_<30
0,30,1.67132,50.802345,18.19,0,True,False,False
1,31,1.81610,61.688562,18.70,0,False,False,True
2,29,1.76276,69.399633,22.33,0,True,False,False
3,28,1.73228,64.410117,21.46,1,False,False,True
4,24,1.72212,65.317301,22.02,1,False,False,True
5,26,1.74498,55.791862,18.32,0,False,True,False
6,22,1.77292,63.956524,20.35,1,False,True,False
7,20,1.78054,61.688562,19.46,1,False,False,True
8,19,1.72466,50.802345,17.08,0,False,False,True
9,31,1.69672,54.431084,18.91,0,True,False,False


In [65]:
os.makedirs("data_clean",exist_ok=True)
df.to_csv('/content/data_clean/data.csv',index=False)

In [61]:
summary


,mean,median,std
Age_yr,32.500000,29.500000,12.860361
Grip_kg,26.000000,27.000000,4.521553
Height_m,1.742440,1.738630,0.042435
Weight_kg,59.828834,61.688562,6.455441
BMI,19.682000,19.185000,1.780972
Frailty_binary,0.400000,0.000000,0.516398


In [63]:
corr = df["Grip_kg"].corr(df["Frailty_binary"])
round(corr,3)

np.float64(-0.476)